Cena:

Piquenique no quintal de uma casa, com um catavento rodando, flores nascendo, enquanto
dentro da casa há uma pessoa andando pela cozinha e sala.

Objetos:

    Internos (Casa - piso de madeira):
    - Sofá                           [X]
    - Televisão                      [X]
    - Ser Humano [Translação]        []
    - Cozinha 
        -Cadeira, Geladeira, Mesa    [X][X][X]

    Externos (Quintal - piso de grama):
    - Flores [Escala]                [X]
    - Arvore                         []
    - Pique Nique                         
        - Mesa, Pão, Cesta           [X][X][X]
    - Cata Vento [Rotação]           [X]


In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shaders.shader_s import Shader



#----------------------(Inicia Janelas + Shaders)-------------------------
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

ourShader = Shader("shaders/vertex_shader.vs", "shaders/fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()


libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init


In [2]:
glEnable(GL_TEXTURE_2D)
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)

global vertices_list
vertices_list = []    
global textures_coord_list
textures_coord_list = []


def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    img = img.convert("RGB")
    
    image_data = img.tobytes("raw", "RGB", 0, -1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)



'''
É possível encontrar, na Internet, modelos .obj cujas faces não sejam triângulos. Nesses casos, precisamos gerar triângulos a partir dos vértices da face.
A função abaixo retorna a sequência de vértices que permite isso. Créditos: Hélio Nogueira Cardoso e Danielle Modesti (SCC0650 - 2024/2).
'''
def circular_sliding_window_of_three(arr):
    if len(arr) == 3:
        return arr
    circular_arr = arr + [arr[0]]
    result = []
    for i in range(len(circular_arr) - 2):
        result.extend(circular_arr[i:i+3])
    return result


def load_obj_and_texture(objFile, texturesList):
    modelo = load_model_from_file(objFile)
    
    ### inserindo vertices do modelo no vetor de vertices
    verticeInicial = len(vertices_list)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, len(vertices_list)))
    faces_visited = []
    for face in modelo['faces']:
        if face[2] not in faces_visited:
            faces_visited.append(face[2])
        for vertice_id in circular_sliding_window_of_three(face[0]):
            vertices_list.append(modelo['vertices'][vertice_id - 1])
        for texture_id in circular_sliding_window_of_three(face[1]):
            if texture_id > 0:
                textures_coord_list.append(modelo['texture'][texture_id - 1])
            else:
                textures_coord_list.append([0.0, 0.0])  # Ou outra coordenada padrão

        
    verticeFinal = len(vertices_list)
    print('Processando modelo {}. Vertice final: {}'.format(objFile, len(vertices_list)))
    
    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    for id in range(len(texturesList)):
        load_texture_from_file(id,texturesList[id])
    
    return verticeInicial, verticeFinal - verticeInicial


def load_obj_and_textures_multi(objFile, material_texture_map):
    """
    Carrega um modelo OBJ e associa múltiplos materiais às suas texturas.
    
    Parameters:
    - objFile: caminho para o arquivo .obj
    - material_texture_map: dicionário { 'nome_material': 'caminho/da/texture.jpg', ... }
    
    Returns:
    - lista de tuplas: (verticeInicial, verticeFinal, texture_id)
    """
    modelo = load_model_from_file(objFile)

    vertice_inicial = len(vertices_list)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, vertice_inicial))

    faces_by_material = {}

    # Organiza as faces por material
    for face in modelo['faces']:
        material = face[2]
        if material not in faces_by_material:
            faces_by_material[material] = []
        faces_by_material[material].append(face)

    textura_ids = {}
    resultados = []

    # Para cada material, processa suas faces e carrega a textura
    for idx, (material, faces) in enumerate(faces_by_material.items()):
        print(f'Processando material: {material}')
        
        # Atribui um texture_id para cada material (usando o índice atual)
        texture_path = material_texture_map.get(material)
        if texture_path is None:
            print(f' Atenção: Material {material} não encontrado no dicionário. Pulando.')
            continue

        textura_ids[material] = idx
        load_texture_from_file(idx, texture_path)

        # Para cada face desse material
        for face in faces:
            for vertice_id in circular_sliding_window_of_three(face[0]):
                vertices_list.append(modelo['vertices'][vertice_id - 1])
            for texture_id in circular_sliding_window_of_three(face[1]):
                textures_coord_list.append(modelo['texture'][texture_id - 1])

        vertice_final = len(vertices_list)
        print(f'Material {material}: vertices {vertice_inicial} a {vertice_final}')
        resultados.append((vertice_inicial, vertice_final, idx))

        vertice_inicial = vertice_final  # atualiza para o próximo material

    return resultados

In [ ]:

material_texture_map = {
    'Drk_Wood': 'obj/sofa/sofa_textures/Color.jpg',
    'Label': 'obj/sofa/sofa_textures/tag.jpg',
    'Orange_Leather_simple': 'obj/sofa/sofa_textures/orange_leather.jpg',
    'UnderMesh': 'obj/sofa/sofa_textures/metallic.png'
}

material_texture_map_flor = {
    'objectSG': 'obj/flower/123.bmp'
}


material_texture_map_fridge = {
    'Galvanized_steel.001': 'obj/refrigerator/back_metal_texture/GalvanizedSteel01_2K_BaseColor.jpg',
    'baked.002': 'obj/refrigerator/color.png',
    'black.002': None  
}

material_texture_map_tv = {
    'V-Tv': 'obj/television/Texture/V-TV_Base_Color.PNG'
}

material_texture_map_table = {
    'Black_Oak_Wood_Veneer': 'obj/table_outside/Texture_maps/c17_wood_02_diff.jpg',
    'Bolt': None,
    'Wood': 'obj/table_outside/Texture_maps/woodcreator_1_basecolor.jpg'
}

material_texture_map_bread = {
    'bread03': 'obj/bread/bread03DiffuseMap.png',
    'bread1': 'obj/bread/bread01DiffuseMap.png',
    'bread2': 'obj/bread/bread02DiffuseMap.png'
}

material_texture_map_wicker = {
    'Wood': 'obj/basket/Textures/WoodFineDark004_COL_2K.jpg'
}

material_texture_map_pinwheel = {
    'mat1': 'obj/pinwheel/mat1.png',
    'mat2': 'obj/pinwheel/mat2.png',
    'mat3': 'obj/pinwheel/mat3.png',
    'mat4': 'obj/pinwheel/mat4.png',
    'mat5': 'obj/pinwheel/mat5.png'
}

material_texture_map_tree = {
    'Lemon_Bark': 'obj/tree/Lemon_Bark_Color.png',
    'Lemon_Branch': 'obj/tree/Lemon_Branch_Color.png'
}


# resultados_sofa = load_obj_and_textures_multi('obj/sofa/sofa.obj', material_texture_map)
# verticeInicial_mesa, quantosVertices_mesa = load_obj_and_texture('obj/table/WoodenTable.obj', ['obj/table/wooden_table_color.png']) 
# verticeInicial_mesa, quantosVertices_mesa = load_obj_and_texture('obj/chair/armchair.obj', ['obj/chair/1.jpg']) 
# resultados_fridge = load_obj_and_textures_multi('obj/refrigerator/fridge_export.obj', material_texture_map_fridge)
# resultados_flor = load_obj_and_textures_multi('obj/flower/Rose.obj', material_texture_map_flor)
# resultados_tv = load_obj_and_textures_multi('obj/television/V-Tv.obj', material_texture_map_tv)
# resultados_table = load_obj_and_textures_multi('obj/table_outside/woodentable.obj', material_texture_map_table)
# resultados_bread = load_obj_and_textures_multi('obj/bread/bread.obj', material_texture_map_bread)
# resultados_pinwheel = load_obj_and_textures_multi('obj/pinwheel/3d-model.obj', material_texture_map_pinwheel)
# resultados_basket = load_obj_and_textures_multi('obj/basket/wicker_basket.obj', material_texture_map_wicker)


resultados_tree = load_obj_and_textures_multi('obj/tree/Lemon_Tree.obj', material_texture_map_tree)

# Função de desenho
def desenha_tree():
    desenha_objeto(resultados_tree, t_x=0.0, t_y=0.0, t_z=0.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0)

# Função de desenho
def desenha_basket():
    desenha_objeto(resultados_basket, t_x=0.0, t_y=0.0, t_z=0.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0)
    
# Função de desenho
def desenha_pinwheel():
    desenha_objeto(resultados_pinwheel, t_x=0.0, t_y=0.0, t_z=0.0,
                   angle=90, r_x=1, r_y=1.0, r_z=1,
                   s_x=0.02, s_y=0.02, s_z=0.02)
    
def desenha_bread():
    desenha_objeto(resultados_bread, t_x=0.0, t_y=0.0, t_z=0.0, 
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0)
    

def desenha_sofa():
    desenha_objeto(resultados_sofa)

def desenha_fridge():
    desenha_objeto(resultados_fridge)

def desenha_flor():
    desenha_objeto(resultados_flor, t_x=-2.0, t_y=0.0, t_z=1.0)

def desenha_mesa():
    global vertices
    # mesa usa um caminho separado, já que não tem resultados_*
    mat_model = model(0, 0, 0, 0, 0, 0, 0, 0.5, 0.5, 0.5)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 0)
    glDrawArrays(GL_TRIANGLES, verticeInicial_mesa, quantosVertices_mesa)

def desenha_tv():
    desenha_objeto(resultados_tv)

def desenha_table():
    desenha_objeto(resultados_table, t_x=0.0, t_y=0.0, t_z=0.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0)
    
    
def desenha_objeto(resultados, t_x=0.0, t_y=0.0, t_z=0.0, 
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0):
    """Função genérica para desenhar objetos"""
    global vertices

    # Calcula a matriz model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    # Desenha cada parte do objeto
    for vertice_inicial, vertice_final, texture_id in resultados:
        if texture_id is not None:
            glBindTexture(GL_TEXTURE_2D, texture_id)
        else:
            glBindTexture(GL_TEXTURE_2D, 0)
        quantos_vertices = vertice_final - vertice_inicial
        glDrawArrays(GL_TRIANGLES, vertice_inicial, quantos_vertices)


Processando modelo obj/tree/Lemon_Tree.obj. Vertice inicial: 0
Processando material: Lemon_Branch
Material Lemon_Branch: vertices 0 a 2460
Processando material: Lemon_Bark
Material Lemon_Bark: vertices 2460 a 7170


In [4]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
       
    # aplicando translacao (terceira operação a ser executada)
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao (segunda operação a ser executada)
    if angle!=0:
        matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala (primeira operação a ser executada)
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)


    # \/ aula sobre projection
    # Testar:
    #mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 40.0)
    #mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 40, 60.0)
    #mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 100.0) #Original
    #mat_projection = glm.perspective(glm.radians(10.0), largura/altura, 0.1, 100.0) # reduzir o ângulo = afastar o centro de projeção = zoom-in em uma pequena região
    #mat_projection = glm.perspective(glm.radians(90.0), largura/altura, 0.1, 100.0) # aumentar o ângulo = aproximar o centro de projeção = zoom-out na cena
    
    mat_projection = np.array(mat_projection)    
    return mat_projection

In [5]:
buffer_VBO = glGenBuffers(2)

vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

In [6]:
#cameraPos   = glm.vec3(0.0,  0.0,  1.0);
#cameraFront = glm.vec3(0.0,  0.0, -1.0);
#cameraUp    = glm.vec3(0.0,  1.0,  0.0);

# camera
cameraPos   = glm.vec3(0.0, 0.0, 3.0)
cameraFront = glm.vec3(0.0, 0.0, -1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

firstMouse = True
yaw   = -90.0	# yaw is initialized to -90.0 degrees since a yaw of 0.0 results in a direction vector pointing to the right so we initially rotate a bit to the left.
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# timing
deltaTime = 0.0	# time between current frame and last frame
lastFrame = 0.0


firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp

    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    
    cameraSpeed = 50 * deltaTime
    if key == glfw.KEY_W and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += cameraSpeed * cameraFront
    
    if key == glfw.KEY_S and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= cameraSpeed * cameraFront
    
    if key == glfw.KEY_A and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == glfw.KEY_D and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        

def framebuffer_size_callback(window, largura, altura):

    # make sure the viewport matches the new window dimensions note that width and 
    # height will be significantly larger than specified on retina displays.
    glViewport(0, 0, largura, altura)

# glfw: whenever the mouse moves, this callback is called
# -------------------------------------------------------
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch
   
    if (firstMouse):

        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos # reversed since y-coordinates go from bottom to top
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 # change this value to your liking
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    # make sure that when pitch is out of bounds, screen doesn't get flipped
    if (pitch > 89.0):
        pitch = 89.0
    if (pitch < -89.0):
        pitch = -89.0

    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# glfw: whenever the mouse scroll wheel scrolls, this callback is called
# ----------------------------------------------------------------------
def scroll_callback(window, xoffset, yoffset):
    global fov

    fov -= yoffset
    if (fov < 1.0):
        fov = 1.0
    if (fov > 45.0):
        fov = 45.0
    
glfw.set_key_callback(window,key_event)
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# tell GLFW to capture our mouse
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

In [7]:
glfw.show_window(window)
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
while not glfw.window_should_close(window):

    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

    glfw.poll_events() 
       
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) 

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    #desenha_mesa()
    #desenha_sofa()
    #desenha_flor()
    #desenha_fridge()
    #desenha_tv()
    #desenha_table()
    #desenha_bread()
    #desenha_pinwheel()
    #desenha_basket()
    desenha_tree()
    glfw.swap_buffers(window)

glfw.terminate()